In [0]:
%sql
CREATE TABLE MAN_CATALOG.MAN_SCHEMA.DELTA_TBL
(
ID INT,
NAME STRING,
CITY STRING
)
USING DELTA
LOCATION 'abfss://mycontainer@mystorageaccountrachit.dfs.core.windows.net/deltalake/delta_tbl';

In [0]:
%sql
ALTER TABLE MAN_CATALOG.MAN_SCHEMA.delta_tbl SET TBLPROPERTIES ('delta.enableDeletionVectors' = False);

In [0]:
%sql
INSERT INTO MAN_CATALOG.MAN_SCHEMA.delta_tbl
VALUES
(1,'AA','DDUN'),
(2,'BB','PUNE'),
(3,'CC','PONDICHERRY');

In [0]:
%sql
DESCRIBE EXTENDED MAN_CATALOG.MAN_SCHEMA.delta_tbl;

In [0]:
%sql
 UPDATE MAN_CATALOG.MAN_SCHEMA.delta_tbl SET CITY = 'CHENNAI' WHERE ID = 2;

Version

In [0]:
%sql
DESCRIBE HISTORY MAN_CATALOG.MAN_SCHEMA.DELTA_TBL;

**Time Travel**

In [0]:
%sql
RESTORE TABLE MAN_CATALOG.MAN_SCHEMA.DELTA_TBL TO VERSION AS OF 2;

In [0]:
%sql
SELECT * FROM man_catalog.man_schema.delta_tbl

### DELETION VECTOR

Deletion vectors are a storage optimization feature you can enable on Delta Lake tables. By default, when a single row in a data file is deleted, the entire Parquet file containing the record must be rewritten. With deletion vectors enabled for the table, DELETE, UPDATE, and MERGE operations use deletion vectors to mark existing rows as removed or changed without rewriting the Parquet file. Subsequent reads on the table resolve the current table state by applying the deletions indicated by deletion vectors to the most recent table version.


In [0]:
%sql
CREATE TABLE MAN_CATALOG.MAN_SCHEMA.DELTA_TBL2
(
  ID INT,
  NAME STRING,
  CITY STRING
)
USING DELTA
LOCATION 'abfss://mycontainer@mystorageaccountrachit.dfs.core.windows.net/deltalake/delta_tbl2';

In [0]:
%sql
INSERT INTO MAN_CATALOG.MAN_SCHEMA.delta_tbl2
VALUES
(1,'AA','DDUN'),
(2,'BB','PUNE'),
(3,'CC','PONDICHERRY');

#### Updates in Delta table

In [0]:
%sql
 UPDATE MAN_CATALOG.MAN_SCHEMA.delta_tbl2 SET CITY = 'CHENNAI' WHERE ID = 2;
 

###Optimise in delta table

In [0]:
%sql
DESCRIBE HISTORY man_catalog.man_schema.delta_tbl2;

Note: We have not applied OPTIMIZE command here and still seeing OPTIMIZE command in the history. Because we have deletion vectors turned on for our delta tables then delta lake will automatically apply OPTIMIZE command on our delta tabl because it can improve the performance. 

In [0]:
%sql
describe history man_catalog.man_schema.delta_tbl;

In [0]:
%sql
SELECT * FROM man_catalog.man_schema.delta_tbl2 VERSION AS OF 1;

##Deep Clone vs Shallow Clone

Shallow clone: only duplicates the metadata of the table being cloned; the data files of the table itself are not copied. These clones are cheaper to create but are  not self-contained and depend on the source from which they were cloned as the source of data. If the files in the source that the clone depends on are removed, for example with VACUUM, a shallow clone may become unusable. Therefore, shallow clones are typically used for short-lived use cases such as testing and experimentation.Shallow clone is only supported for the MANAGED table type

Deep clone: A deep clone makes a full copy of the metadata and data files of the table being cloned. In that sense it is similar in functionality to copying with a CTAS command- but it is simpler to specify. Any change in the source table will not be affect the Deep cloned table

#### DEEP CLONE

In [0]:
%sql
CREATE TABLE MAN_CATALOG.MAN_SCHEMA.DEEP_CLONE
DEEP CLONE MAN_CATALOG.MAN_SCHEMA.DELTA_TBL2;

DESCRIBE HISTORY

In [0]:
%sql
DESCRIBE HISTORY MAN_CATALOG.MAN_SCHEMA.DEEP_CLONE;

EXTENDED  INFORMATION  (check location)

In [0]:
%sql
DESCRIBE EXTENDED MAN_CATALOG.MAN_SCHEMA.DEEP_CLONE;

CREATING EXTERNAL DEEP CLONE

In [0]:
%sql
CREATE TABLE MAN_CATALOG.MAN_SCHEMA.ext_DEEP_CLONE
DEEP CLONE MAN_CATALOG.MAN_SCHEMA.DELTA_TBL2
LOCATION 'abfss://mycontainer@mystorageaccountrachit.dfs.core.windows.net/deltalake/deep_clone/';

CREATING DEEP CLONE USING TIME TRAVEL

In [0]:
%sql
CREATE TABLE MAN_CATALOG.MAN_SCHEMA.DEEP_CLONE_TIMETRAVEL
CLONE MAN_CATALOG.MAN_SCHEMA.DELTA_TBL2 VERSION AS OF 3;

### SHALLOW CLONE

In [0]:
%sql
CREATE TABLE MAN_CATALOG.MAN_SCHEMA.SHALLOW_CLONE_TBL
SHALLOW CLONE MAN_CATALOG.MAN_SCHEMA.MAN_TABLE;

In [0]:
%sql
DESCRIBE EXTENDED MAN_CATALOG.MAN_SCHEMA.SHALLOW_CLONE_TBL;

In [0]:
%sql
SELECT * FROM man_catalog.man_schema.shallow_clone_tbl;

In [0]:
%sql
SELECT * FROM man_catalog.man_schema.man_table;
INSERT INTO man_catalog.man_schema.man_table VALUES (100,'ASD'),(200,'SGD');

In [0]:
%sql
SELECT * FROM man_catalog.man_schema.man_table;
SELECT * FROM man_catalog.man_schema.shallow_clone_tbl;

In [0]:
%sql
DESCRIBE EXTENDED man_catalog.man_schema.shallow_clone_tbl;

In [0]:
%sql
DESCRIBE HISTORY man_catalog.man_schema.shallow_clone_tbl;